In [1]:
# Import necessary packages
from sqlalchemy import create_engine, inspect
from sqlalchemy.exc import SQLAlchemyError
from langchain import LLMChain
from langchain_ollama.llms import OllamaLLM
from langchain.prompts import PromptTemplate
from langchain_core.vectorstores import VectorStoreRetriever
from langchain.vectorstores import FAISS
from langchain_openai import ChatOpenAI
open_ai_key = "
"
db_user = "root"
db_password = "password"
db_host = "localhost"
db_name = "otl_temp"

# 1. Set up the database connection
DATABASE_URL = f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}"
engine = create_engine(DATABASE_URL)
inspector = inspect(engine)

# 2. Retrieve and format database schema information
def get_database_schema():
    schema_info = {}
    for table_name in inspector.get_table_names():
        columns = inspector.get_columns(table_name)
        schema_info[table_name] = [column['name'] for column in columns]
    
    schema_str = "Database Schema:\n"
    for table, columns in schema_info.items():
        schema_str += f"Table: {table}\nColumns: {', '.join(columns)}\n\n"
    return schema_info, schema_str

# Get the schema information and schema string
schema_info, database_schema = get_database_schema()  # Convert schema information into a dictionary and a string
table_list_import = [
    'import_pi', 'supplier', 'imp_lc_pi', 'import_lc', 'imp_lc_status', 'import_ci', 'import_flow_status',
    'import_shipment', 'import_permit', 'custom_clearance', 'supplier_payment', 
    'inventory', 'exp_pi_prod_dtl', 'export_pi', 'exp_pi_rev', 'exp_pi_rev_dtl', 
    'customer', 'marketing_person', 'product', 'prod_raw_material', 
    'product_req_from_mktp', 'imp_ci_prod_dtl', 'imp_pi_prod_dtl', 
    'inv_action_type', 'inv_action_type_section', 'ImportLCStatus', 
    'ImportPI', 'ImportLC', 'ImportShipmentType', 'composition','country','supp_org_type'
]
table_list = [
    'import_pi', 'supplier','country'
]
filtered_schema_info = {table: columns for table, columns in schema_info.items() if table in table_list_import}

# Corrected list of schema documents for retrieval
schema_documents = [f"Table: {table} Columns: {', '.join(columns)}" for table, columns in filtered_schema_info.items()]

# filtered_schema_info
#schema_documents = [f"Table: {table} Columns: {', '.join(columns)}" for table, columns in filtered_schema_info.items()]
# Print the formatted schema information
#print((schema_documents))


SyntaxError: unterminated string literal (detected at line 10) (3368936272.py, line 10)

In [163]:
table_information = """
Table: import_pi
    - id: INT, Primary Key. Unique identifier for each import Proforma Invoice.
    - unique_id: VARCHAR. Unique identifier for the PI within the import process.
    - imp_pi_number: VARCHAR. Import PI number.
    - revision_date: DATE. Date of revision for the PI.
    - imp_pi_rev_type_id: INT. Type of revision for the PI.
    - imp_pi_date: DATE. Date of the PI.
    - supplier_id: INT, Foreign Key (supplier.id). References the supplier for the PI.
    - cfr_value: DECIMAL. Cost and Freight value in the PI.
    - freight_charge: DECIMAL. Freight charges in the PI.
    - fob: DECIMAL. Free on Board value in the PI.
    - etd_date_china: DATE. Estimated Time of Departure from China.
    - imp_pi_status_id: INT. Status ID of the PI.
    - cancellation_date: DATE. Date of cancellation, if applicable.
    - cancellation_reason: VARCHAR. Reason for cancellation.
    - import_flow_status_id: INT. Status ID of the import process.
    - isCompensationAdjusted: BOOLEAN. Indicates if compensation has been adjusted.
    - total_revision_count: INT. Total number of revisions.
    - submitted_to_commercial: BOOLEAN. Indicates if the PI has been submitted to the commercial department.
    - created_at: TIMESTAMP. Record creation date and time.
    - created_by: INT. User ID of the record creator.
    - updated_at: TIMESTAMP. Last update date and time.
    - updated_by: INT. User ID of the last updater.

Table: supplier
    - id: INT, Primary Key. Unique identifier for each supplier.
    - name: VARCHAR. Full name of the supplier.
    - short_name: VARCHAR. Short name or abbreviation of the supplier.
    - address: TEXT. Address of the supplier.
    - active: BOOLEAN. Indicates if the supplier is active.
    - contact_person: VARCHAR. Name of the contact person.
    - contact_number: VARCHAR. Phone number of the contact person.
    - email: VARCHAR. Email address of the contact person.
    - website: VARCHAR. Supplier’s website URL.
    - province: VARCHAR. Province or state of the supplier.
    - country_id: INT. Country ID where the supplier is located.
    - port: VARCHAR. Port used by the supplier for shipping.
    - enlisted_date: DATE. Date when the supplier was enlisted.
    - no_of_factories: INT. Number of factories owned by the supplier.
    - total_employee: INT. Total number of employees in the supplier’s organization.
    - annual_revenue: DECIMAL. Annual revenue of the supplier.
    - supp_org_type_id: INT. Type of supplier organization.
    - created_by: INT. User ID of the record creator.
    - created_at: TIMESTAMP. Record creation date and time.
    - updated_at: TIMESTAMP. Last update date and time.
    - updated_by: INT. User ID of the last updater.

Table: imp_lc_pi
    - id: INT, Primary Key. Unique identifier for each LC and PI linkage.
    - import_lc_id: INT, Foreign Key (import_lc.id). References the import LC.
    - import_pi_id: INT, Foreign Key (import_pi.id). References the import PI.
    - maturity_date: DATE. Maturity date of the LC.
    - payment_date: DATE. Payment date for the LC.
    - previous_cfr: DECIMAL. Previous Cost and Freight value.
    - cfr_value: DECIMAL. Cost and Freight value in the LC.
    - freight_charge: DECIMAL. Freight charges in the LC.
    - fob: DECIMAL. Free on Board value in the LC.
    - import_pi_unique_id: VARCHAR. Unique identifier for the PI within the LC.
    - created_at: TIMESTAMP. Record creation date and time.
    - created_by: INT. User ID of the record creator.
    - deleted_at: TIMESTAMP. Deletion date and time, if applicable.
    - deleted_by: INT. User ID of the person who deleted the record, if applicable.

Table: import_lc
    - id: INT, Primary Key. Unique identifier for each import LC.
    - unique_id: VARCHAR. Unique identifier for the LC.
    - supplier_id: INT, Foreign Key (supplier.id). References the supplier associated with the LC.
    - lc_opening_date: DATE. Date when the LC was opened.
    - lc_number: VARCHAR. Letter of Credit number.
    - imp_lc_tenure_id: INT. Tenure ID of the LC.
    - tolerance_level_pct: DECIMAL. Tolerance level percentage.
    - tolerance_level_amount: DECIMAL. Tolerance level amount.
    - lc_value: DECIMAL. Total value of the LC.
    - last_ship_date: DATE. Last date of shipment under the LC.
    - imp_lc_status_id: INT, Foreign Key (imp_lc_status.id). References the status of the LC.
    - lc_balance_value: DECIMAL. Remaining balance value of the LC.
    - cancellation_date: DATE. Date of cancellation, if applicable.
    - cancellation_reason: VARCHAR. Reason for cancellation.
    - imp_shipment_mode_id: INT. Shipment mode ID for the LC.
    - lc_under_id: INT. ID of the entity under which the LC is created.
    - total_cfr_value: DECIMAL. Total Cost and Freight value.
    - total_freight_charge: DECIMAL. Total freight charges.
    - total_fob: DECIMAL. Total Free on Board value.
    - total_revision_count: INT. Total number of revisions.
    - import_flow_status_id: INT. Status ID of the import process.
    - created_by: INT. User ID of the record creator.
    - revision_date: DATE. Date of the last revision.
    - created_at: TIMESTAMP. Record creation date and time.
    - updated_at: TIMESTAMP. Last update date and time.
    - updated_by: INT. User ID of the last updater.

Table: imp_lc_status
    - id: INT, Primary Key. Unique identifier for each LC status.
    - imp_lc_status: VARCHAR. Status name of the import LC (e.g., Open, Closed, Cancelled).

Table: import_ci
    - id: INT, Primary Key. Unique identifier for each import commercial invoice.
    - supplier_id: INT, Foreign Key (supplier.id). References the supplier providing the goods.
    - import_lc_id: INT, Foreign Key (import_lc.id). References the related Letter of Credit.
    - invoice_no: VARCHAR. Invoice number.
    - ci_date: DATE. Date of the commercial invoice.
    - total: DECIMAL. Total amount of the invoice.
    - cfr_value: DECIMAL. Cost and Freight value in the invoice.
    - freight_charge: DECIMAL. Freight charge in the invoice.
    - fob: DECIMAL. Free on Board value in the invoice.
    - import_flow_status_id: INT. Status ID of the import process.
    - imp_ci_status_id: INT. Status ID of the commercial invoice.
    - import_qc_inspection_status: INT. Quality Control inspection status ID.
    - cancellation_date: DATE. Date of cancellation, if applicable.
    - cancellation_reason: VARCHAR. Reason for cancellation.
    - created_at: TIMESTAMP. Record creation date and time.
    - created_by: INT. User ID of the record creator.
    - updated_at: TIMESTAMP. Last update date and time.
    - updated_by: INT. User ID of the last updater.

Table: import_shipment
    - id: INT, Primary Key. Unique identifier for each import shipment.
    - supplier_id: INT, Foreign Key (supplier.id). References the supplier for the shipment.
    - import_ci_id: INT, Foreign Key (import_ci.id). References the import commercial invoice.
    - actual_ship_date: DATE. Actual shipment date.
    - eta_ctg_date: DATE. Estimated time of arrival at Chittagong.
    - imp_shipment_type_id: INT. ID of the shipment type.
    - imp_shipment_mode_id: INT. ID of the shipment mode.
    - bl_no_or_awb_no: VARCHAR. Bill of Lading or Air Waybill number.
    - container_no: VARCHAR. Container number.
    - courier_no: VARCHAR. Courier tracking number.
    - doc_received_by_bank_date: DATE. Date when documents were received by the bank.
    - doc_received_from_bank_date: DATE. Date when documents were received from the bank.
    - ratings_on_shipment: DECIMAL. Rating given to the shipment.
    - lead_time_for_shipment: INT. Lead time in days for the shipment.
    - shipment_delayed_no_of_days: INT. Number of days the shipment was delayed.
    - import_flow_status_id: INT. Status ID of the import flow process.
    - imp_shipment_status_id: INT. Status ID of the shipment.
    - cancellation_date: DATE. Date of cancellation, if applicable.
    - cancellation_reason: VARCHAR. Reason for cancellation.
    - created_at: TIMESTAMP. Record creation date and time.
    - created_by: INT. User ID of the record creator.
    - updated_at: TIMESTAMP. Last update date and time.
    - updated_by: INT. User ID of the last updater.

Table: import_permit
    - id: INT, Primary Key. Unique identifier for each import permit.
    - supplier_id: INT, Foreign Key (supplier.id). References the supplier for the permit.
    - import_ci_id: INT, Foreign Key (import_ci.id). References the import commercial invoice.
    - import_shipment_id: INT, Foreign Key (import_shipment.id). References the import shipment.
    - doc_receive_date_by_depz: DATE. Date when documents were received by DEPZ.
    - ip_issue_date_from_depz: DATE. Date when the import permit was issued from DEPZ.
    - import_flow_status_id: INT. Status ID of the import flow process.
    - imp_permit_status_id: INT. Status ID of the import permit.
    - cancellation_date: DATE. Date of cancellation, if applicable.
    - cancellation_reason: VARCHAR. Reason for cancellation.
    - import_permit_type_id: INT. Type ID of the import permit.
    - ip_number: VARCHAR. Import permit number.
    - ip_sending_date_to_ctg: DATE. Date when the import permit was sent to Chittagong.
    - created_at: TIMESTAMP. Record creation date and time.
    - created_by: INT. User ID of the record creator.
    - updated_at: TIMESTAMP. Last update date and time.
    - updated_by: INT. User ID of the last updater.

Table: custom_clearance
    - id: INT, Primary Key. Unique identifier for each custom clearance record.
    - supplier_id: INT, Foreign Key (supplier.id). References the supplier of the imported goods.
    - import_ci_id: INT, Foreign Key (import_ci.id). References the import commercial invoice related to the clearance.
    - ip_doc_receive_date_by_ctg: DATE. Date when the import permit document was received by the Chittagong office.
    - indent_date_by_ctg: DATE. Date of indent issuance by Chittagong office.
    - indent_date_by_dhaka: DATE. Date of indent issuance by Dhaka office.
    - goods_released_date_from_custom: DATE. Date when goods were released from customs.
    - est_in_house_date: DATE. Estimated date when goods will be in-house.
    - bill_of_entry_no: VARCHAR. Bill of Entry number for the customs.
    - bill_of_entry_date: DATE. Date when the Bill of Entry was filed.
    - import_flow_status_id: INT. Status ID of the import flow process.
    - cust_clearance_status_id: INT. Status ID of the customs clearance process.
    - cancellation_date: DATE. Date of cancellation, if any.
    - cancellation_reason: VARCHAR. Reason for cancellation, if applicable.
    - import_permit_type_id: INT. Type of import permit used.
    - created_at: TIMESTAMP. Record creation date and time.
    - created_by: INT. User ID of the record creator.
    - updated_at: TIMESTAMP. Last update date and time.
    - updated_by: INT. User ID of the last updater.

Table: supplier_payment
    - id: INT, Primary Key. Unique identifier for each supplier payment.
    - supplier_id: INT, Foreign Key (supplier.id). References the supplier receiving the payment.
    - import_ci_id: INT, Foreign Key (import_ci.id). References the related import commercial invoice.
    - invoice_payment: VARCHAR. Payment details of the invoice.
    - paid_on_date: DATE. Date when the payment was made.
    - payment_amount: DECIMAL. Amount paid to the supplier.
    - supp_payment_status_id: INT. Status of the supplier payment.
    - created_at: TIMESTAMP. Record creation date and time.
    - created_by: INT. User ID of the record creator.
    - updated_at: TIMESTAMP. Last update date and time.
    - updated_by: INT. User ID of the last updater.

Table: inventory
    - id: INT, Primary Key. Unique identifier for each inventory record.
    - transfer_id: INT. Transfer ID associated with the inventory.
    - lot_number: VARCHAR. Lot number for the inventory.
    - inventory_date: DATE. Date of the inventory entry.
    - ref_no: VARCHAR. Reference number for the inventory.
    - prod_ref_no: VARCHAR. Production reference number for the inventory.
    - action_type_id: INT, Foreign Key (inv_action_type.id). References the type of action (e.g., receipt, dispatch).
    - import_pi_id: INT, Foreign Key (import_pi.id). References the related import PI.
    - import_ci_id: INT, Foreign Key (import_ci.id). References the related import commercial invoice.
    - supplier_id: INT, Foreign Key (supplier.id). References the supplier associated with the inventory.
    - export_booking_id: INT. ID of the export booking associated with the inventory.
    - export_pi_id: INT, Foreign Key (export_pi.id). References the related export PI.
    - prod_requisition_id: INT. ID of the production requisition associated with the inventory.
    - del_challan_id: INT. ID of the delivery challan.
    - del_requisition_number: VARCHAR. Delivery requisition number.
    - delivery_person: VARCHAR. Name of the person responsible for delivery.
    - delivery_person_contact: VARCHAR. Contact information for the delivery person.
    - mark_as_fully_delivered_date: DATE. Date when marked as fully delivered.
    - mark_as_fully_delivered: BOOLEAN. Indicates if the inventory is fully delivered.
    - receive_url: VARCHAR. URL to receive the inventory.
    - receive_url_status: VARCHAR. Status of the receive URL.
    - mark_as_full_shipment: BOOLEAN. Indicates if the inventory is a full shipment.
    - mark_as_full_shipment_date: DATE. Date when marked as full shipment.
    - customer_id: INT, Foreign Key (customer.id). References the customer associated with the inventory.
    - is_packing_only: BOOLEAN. Indicates if the inventory is for packing only.
    - created_at: TIMESTAMP. Record creation date and time.
    - created_by: INT. User ID of the record creator.
    - updated_at: TIMESTAMP. Last update date and time.
    - updated_by: INT. User ID of the last updater.
    - deleted_at: TIMESTAMP. Date of deletion, if applicable.
    - deleted_by: INT. User ID of the person who deleted the record, if applicable.

Table: exp_pi_prod_dtl
    - id: INT, Primary Key. Unique identifier for each export PI product detail.
    - export_pi_id: INT, Foreign Key (export_pi.id). References the export Proforma Invoice.
    - unique_id: VARCHAR. Unique identifier for the product detail within the PI.
    - product_id: INT, Foreign Key (product.id). References the product being exported.
    - product_sticker_id: INT. ID for product sticker details.
    - style_id: INT. ID for the style associated with the product.
    - po_number_id: INT. ID for the purchase order number.
    - unit_id: INT. ID of the unit of measure.
    - qty: DECIMAL. Quantity of the product in the PI.
    - unit_price: DECIMAL. Unit price of the product.
    - amount: DECIMAL. Total amount for the quantity specified.
    - unit_spl: VARCHAR. Special unit specification, if any.
    - spl_amount: DECIMAL. Special amount calculation based on the unit_spl.
    - actual_value: DECIMAL. Actual value of the product.
    - net_weight: DECIMAL. Net weight of the product.
    - accessories_weight: DECIMAL. Weight of accessories included with the product.
    - gross_weight: DECIMAL. Gross weight of the product.
    - created_at: TIMESTAMP. Record creation date and time.
    - created_by: INT. User ID of the record creator.
    - deleted_at: TIMESTAMP. Deletion date and time, if applicable.
    - deleted_by: INT. User ID of the person who deleted the record, if applicable.

Table: export_pi
    - id: INT, Primary Key. Unique identifier for each export Proforma Invoice.
    - exp_pi_type_id: INT. ID of the export PI type.
    - is_merge_pi: BOOLEAN. Indicates if the PI is merged.
    - remarks: TEXT. Additional remarks or comments.
    - pi_number: VARCHAR. Proforma Invoice number.
    - pi_date: DATE. Date of the Proforma Invoice.
    - export_booking_id: INT. ID of the export booking.
    - bdt_rate: DECIMAL. BDT rate for the PI.
    - exp_organization_id: INT. ID of the exporting organization.
    - marketing_person_id: INT. ID of the marketing person handling the PI.
    - customer_id: INT, Foreign Key (customer.id). References the customer.
    - brand_id: INT. ID of the brand associated with the PI.
    - style_id: INT. ID of the style associated with the PI.
    - po_number_id: INT. ID of the purchase order number.
    - style_po_no: VARCHAR. Style and PO number combined.
    - expected_delivery_date: DATE. Expected date of delivery.
    - sales_mode_id: INT. ID of the sales mode.
    - exp_lc_tenure_id: INT. ID of the export LC tenure.
    - percent_in_spl: DECIMAL. Percentage in special pricing.
    - exp_pi_status_id: INT. ID of the export PI status.
    - export_flow_status_id: INT. ID of the export flow status.
    - export_order_status_id: INT. ID of the export order status.
    - cancellation_date: DATE. Date of cancellation, if applicable.
    - cancellation_reason: VARCHAR. Reason for cancellation.
    - unit_id: INT. ID of the unit of measure.
    - total_pi_value: DECIMAL. Total value of the PI.
    - total_spl_value: DECIMAL. Total special value of the PI.
    - total_actual_value: DECIMAL. Total actual value of the PI.
    - total_weight: DECIMAL. Total weight of the PI.
    - total_gross_weight: DECIMAL. Total gross weight of the PI.
    - bank_account_id: INT. ID of the bank account used.
    - applicant: VARCHAR. Name of the applicant.
    - hs_code: VARCHAR. Harmonized System code for international trade.
    - exp_payment_mode_id: INT. ID of the export payment mode.
    - exp_charge_type_id: INT. ID of the export charge type.
    - amount_type_id: INT. ID of the amount type.
    - charge_percentage: DECIMAL. Percentage of the charge applied.
    - charge_value: DECIMAL. Value of the charge applied.
    - discount_percentage: DECIMAL. Discount percentage applied.
    - discount_value: DECIMAL. Discount value applied.
    - after_discount_total: DECIMAL. Total value after discount.
    - is_sales_returned: BOOLEAN. Indicates if the sales have been returned.
    - created_at: TIMESTAMP. Record creation date and time.
    - created_by: INT. User ID of the record creator.
    - updated_at: TIMESTAMP. Last update date and time.
    - updated_by: INT. User ID of the last updater.

Table: exp_pi_rev
    - id: INT, Primary Key. Unique identifier for each export PI revision.
    - serial: INT. Serial number of the revision.
    - reason: TEXT. Reason for the revision.
    - revision_date: DATE. Date when the revision was made.
    - export_pi_id: INT, Foreign Key (export_pi.id). References the export Proforma Invoice being revised.
    - unit_id: INT. ID of the unit of measure.
    - percent_in_spl: DECIMAL. Percentage of special pricing.
    - total_pi_value: DECIMAL. Total PI value after revision.
    - total_spl_value: DECIMAL. Total special value after revision.
    - total_actual_value: DECIMAL. Total actual value after revision.
    - total_weight: DECIMAL. Total weight after revision.
    - total_gross_weight: DECIMAL. Total gross weight after revision.
    - exp_charge_type_id: INT. ID of the charge type in the export.
    - amount_type_id: INT. ID of the amount type in the export.
    - charge_percentage: DECIMAL. Percentage of the charge applied.
    - charge_value: DECIMAL. Value of the charge applied.
    - discount_percentage: DECIMAL. Discount percentage applied.
    - discount_value: DECIMAL. Discount value applied.
    - after_discount_total: DECIMAL. Total value after discount.
    - created_at: TIMESTAMP. Record creation date and time.
    - created_by: INT. User ID of the record creator.
    - updated_at: TIMESTAMP. Last update date and time.
    - updated_by: INT. User ID of the last updater.

Table: exp_pi_rev_dtl
    - id: INT, Primary Key. Unique identifier for each revision detail.
    - export_pi_id: INT, Foreign Key (export_pi.id). References the export Proforma Invoice.
    - exp_pi_rev_id: INT, Foreign Key (exp_pi_rev.id). References the export PI revision.
    - unique_id: VARCHAR. Unique identifier for the revision detail.
    - product_id: INT, Foreign Key (product.id). References the product being revised.
    - product_sticker_id: INT. ID for product sticker details.
    - style_id: INT. ID for the style associated with the product.
    - po_number_id: INT. ID for the purchase order number.
    - unit_id: INT. ID of the unit of measure.
    - qty: DECIMAL. Quantity of the product in the revised PI.
    - unit_price: DECIMAL. Unit price of the product.
    - amount: DECIMAL. Total amount for the quantity specified.
    - unit_spl: VARCHAR. Special unit specification, if any.
    - spl_amount: DECIMAL. Special amount calculation based on the unit_spl.
    - actual_value: DECIMAL. Actual value of the product.
    - net_weight: DECIMAL. Net weight of the product.
    - accessories_weight: DECIMAL. Weight of accessories included with the product.
    - gross_weight: DECIMAL. Gross weight of the product.
    - created_at: TIMESTAMP. Record creation date and time.
    - created_by: INT. User ID of the record creator.

Table: customer
    - id: INT, Primary Key. Unique identifier for each customer.
    - customer_name: VARCHAR. Full name of the customer.
    - customer_short_name: VARCHAR. Short name or abbreviation of the customer.
    - store_officer_id: INT. ID of the store officer responsible for this customer.
    - address: TEXT. Customer’s address.
    - active: BOOLEAN. Indicates if the customer is active.
    - customer_type_id: INT. Type or category of the customer.
    - contact_person: VARCHAR. Name of the contact person.
    - contact_number: VARCHAR. Phone number of the contact person.
    - email: VARCHAR. Email address of the contact person.
    - website: VARCHAR. Customer’s website URL.
    - group_name_id: INT. ID for grouping customers under a specific group.
    - division_id: INT. ID for the division under which the customer falls.
    - zone_id: INT. ID for the zone under which the customer falls.
    - name_of_md: VARCHAR. Name of the Managing Director.
    - bank_id: INT. Bank ID associated with the customer.
    - no_of_production_lines: INT. Number of production lines owned by the customer.
    - business_with_oil_since: YEAR. Year since the customer has been in business with oil.
    - no_of_merchandiser: INT. Number of merchandisers employed by the customer.
    - no_of_brands: INT. Number of brands under the customer.
    - prod_capacity_per_month: DECIMAL. Monthly production capacity.
    - party_bond_no: VARCHAR. Party bond number of the customer.
    - vat_reg_no: VARCHAR. VAT registration number.
    - bin_no: VARCHAR. Business Identification Number.
    - tin_no: VARCHAR. Tax Identification Number.
    - erc_no: VARCHAR. Export Registration Certificate number.
    - irc_no: VARCHAR. Import Registration Certificate number.
    - exp_no: VARCHAR. Export permit number.
    - boi_no: VARCHAR. Board of Investment number.
    - sap_no: VARCHAR. SAP identification number.
    - store_officer_name: VARCHAR. Name of the store officer.
    - store_officer_mobile: VARCHAR. Mobile number of the store officer.
    - store_officer_email: VARCHAR. Email address of the store officer.
    - created_at: TIMESTAMP. Record creation date and time.
    - created_by: INT. User ID of the record creator.
    - updated_at: TIMESTAMP. Last update date and time.
    - updated_by: INT. User ID of the last updater.

Table: marketing_person
    - id: INT, Primary Key. Unique identifier for each marketing person.
    - person_name: VARCHAR. Full name of the marketing person.
    - employee_id: VARCHAR. Employee ID of the marketing person.
    - designation_id: INT. ID of the designation held by the marketing person.
    - short_name: VARCHAR. Short name or abbreviation of the marketing person.
    - mobile_no: VARCHAR. Mobile number of the marketing person.
    - email: VARCHAR. Email address of the marketing person.
    - joining_date: DATE. Date of joining.
    - experience_in_year_prev: DECIMAL. Years of prior experience.
    - academic_qualification: VARCHAR. Academic qualification of the person.
    - major_subject: VARCHAR. Major subject studied by the marketing person.
    - created_at: TIMESTAMP. Record creation date and time.
    - created_by: INT. User ID of the record creator.
    - updated_at: TIMESTAMP. Last update date and time.
    - updated_by: INT. User ID of the last updater.

Table: product
    - id: INT, Primary Key. Unique identifier for each product.
    - composition_id: INT. ID of the composition of the product.
    - article: VARCHAR. Article number of the product.
    - product_type_id: INT. Type or category of the product.
    - active: BOOLEAN. Indicates if the product is active.
    - purchase: BOOLEAN. Indicates if the product can be purchased.
    - sell: BOOLEAN. Indicates if the product can be sold.
    - produce: BOOLEAN. Indicates if the product can be produced.
    - convert: BOOLEAN. Indicates if the product can be converted.
    - use: BOOLEAN. Indicates if the product can be used.
    - scrap: BOOLEAN. Indicates if the product can be scrapped.
    - initial_reorder_level: DECIMAL. Initial reorder level for the product.
    - unit_id: INT. ID of the unit of measure.
    - sales_unit_id: INT. ID of the sales unit of measure.
    - hs_code: VARCHAR. Harmonized System code for international trade.
    - color_id: INT. ID of the color of the product.
    - width: DECIMAL. Width of the product.
    - width_m: DECIMAL. Width of the product in meters.
    - roll_size: DECIMAL. Size of the roll of the product.
    - gsm: DECIMAL. GSM (grams per square meter) of the product.
    - accessories_weight: DECIMAL. Weight of accessories included with the product.
    - min_qty_to_consider_undercutting: DECIMAL. Minimum quantity to consider for undercutting.
    - max_commission_per_yard: DECIMAL. Maximum commission per yard for the product.
    - initial_markup: DECIMAL. Initial markup for the product.
    - picture_path: VARCHAR. File path to the product image.
    - created_at: TIMESTAMP. Record creation date and time.
    - created_by: INT. User ID of the record creator.
    - updated_at: TIMESTAMP. Last update date and time.
    - updated_by: INT. User ID of the last updater.

Table: prod_raw_material
    - id: INT, Primary Key. Unique identifier for each raw material record.
    - product_id: INT, Foreign Key (product.id). References the product that uses the raw material.
    - conversion_ratio: DECIMAL. Conversion ratio for the raw material to product.
    - raw_mat_prod_id: INT. ID of the raw material product.
    - unit_id: INT. ID of the unit of measure.
    - deleted_by: INT. User ID of the person who deleted the record, if applicable.
    - deleted_at: TIMESTAMP. Deletion date and time, if applicable.
    - created_by: INT. User ID of the record creator.
    - created_at: TIMESTAMP. Record creation date and time.

Table: product_req_from_mktp
    - id: INT, Primary Key. Unique identifier for each product request.
    - marketing_person_id: INT, Foreign Key (marketing_person.id). References the marketing person making the request.
    - product_req_from_mktp_date: DATE. Date of the product request.
    - projection_year: YEAR. Year of the projection.
    - projection_month: INT. Month of the projection.
    - customer_id: INT, Foreign Key (customer.id). References the customer for whom the product is requested.
    - brand_id: INT. ID of the brand for which the product is requested.
    - product_id: INT, Foreign Key (product.id). References the product being requested.
    - unit_id: INT. ID of the unit of measure.
    - quantity: DECIMAL. Quantity of the product requested.
    - quantity_yd: DECIMAL. Quantity of the product in yards.
    - created_at: TIMESTAMP. Record creation date and time.
    - created_by: INT. User ID of the record creator.
    - updated_at: TIMESTAMP. Last update date and time.
    - updated_by: INT. User ID of the last updater.
    - deleted_at: TIMESTAMP. Deletion date and time, if applicable.
    - deleted_by: INT. User ID of the person who deleted the record, if applicable.

Table: imp_ci_prod_dtl
    - id: INT, Primary Key. Unique identifier for each import CI product detail.
    - imp_ci_lc_pi_id: INT. ID linking to the LC and PI details.
    - import_ci_id: INT, Foreign Key (import_ci.id). References the import commercial invoice.
    - product_id: INT, Foreign Key (product.id). References the product being imported.
    - unit_id: INT. ID of the unit of measure.
    - invoice_quantity: DECIMAL. Quantity listed in the invoice.
    - price_per_unit: DECIMAL. Price per unit of the product.
    - imp_shipment_unit_id: INT. ID of the shipment unit.
    - total: DECIMAL. Total cost for the quantity specified.
    - shipment_quantity: DECIMAL. Quantity shipped.
    - weight_kg_net: DECIMAL. Net weight in kilograms.
    - weight_kg_gross: DECIMAL. Gross weight in kilograms.
    - created_at: TIMESTAMP. Record creation date and time.
    - created_by: INT. User ID of the record creator.
    - deleted_at: TIMESTAMP. Deletion date and time, if applicable.
    - deleted_by: INT. User ID of the person who deleted the record, if applicable.

Table: imp_pi_prod_dtl
    - id: INT, Primary Key. Unique identifier for each import PI product detail.
    - import_pi_id: INT, Foreign Key (import_pi.id). References the import PI.
    - product_id: INT, Foreign Key (product.id). References the product being imported.
    - unit_id: INT. ID of the unit of measure.
    - quantity: DECIMAL. Quantity specified in the PI.
    - last_quotation_price: DECIMAL. Last quoted price for the product.
    - price_per_unit: DECIMAL. Price per unit of the product.
    - total: DECIMAL. Total cost for the quantity specified.
    - created_at: TIMESTAMP. Record creation date and time.
    - created_by: INT. User ID of the record creator.
    - deleted_at: TIMESTAMP. Deletion date and time, if applicable.
    - deleted_by: INT. User ID of the person who deleted the record, if applicable.

Table: inv_action_type
    - id: INT, Primary Key. Unique identifier for each inventory action type.
    - action_type: VARCHAR. Type of action (e.g., Received, Dispatched).
    - action_type_section_id: INT, Foreign Key (inv_action_type_section.id). References the section of the action type.
    - rank: INT. Rank or priority of the action type.

Table: inv_action_type_section
    - id: INT, Primary Key. Unique identifier for each inventory action type section.
    - action_type_section: VARCHAR. Section name for the action type (e.g., Inbound, Outbound).

Table: ImportLCStatus
    - id: INT, Primary Key. Unique identifier for each import LC status.
    - status_name: VARCHAR. Name of the LC status (e.g., Open, Closed, Cancelled).

Table: ImportPI
    - id: INT, Primary Key. Unique identifier for each import PI.
    - pi_number: VARCHAR. PI number for the import.
    - supplier_id: INT, Foreign Key (supplier.id). References the supplier associated with the PI.
    - status_id: INT, Foreign Key (imp_pi_status.id). Status of the import PI.
    - created_at: TIMESTAMP. Record creation date and time.
    - created_by: INT. User ID of the record creator.
    - updated_at: TIMESTAMP. Last update date and time.
    - updated_by: INT. User ID of the last updater.

Table: ImportLC
    - id: INT, Primary Key. Unique identifier for each import LC.
    - lc_number: VARCHAR. LC number for the import.
    - supplier_id: INT, Foreign Key (supplier.id). References the supplier associated with the LC.
    - lc_status_id: INT, Foreign Key (ImportLCStatus.id). Status of the import LC.
    - created_at: TIMESTAMP. Record creation date and time.
    - created_by: INT. User ID of the record creator.
    - updated_at: TIMESTAMP. Last update date and time.
    - updated_by: INT. User ID of the last updater.

Table: ImportShipmentType
    - id: INT, Primary Key. Unique identifier for each import shipment type.
    - shipment_type_name: VARCHAR. Name of the shipment type (e.g., Sea, Air, Land).

Table: composition
    - id: INT, Primary Key. Unique identifier for each composition record.
    - name: VARCHAR. Name of the composition (e.g., Cotton, Polyester).
    - description: TEXT. Description of the composition.
"""

details_information = str(schema_documents) + "\n" + database_info

In [164]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

vectorstore = FAISS.from_texts(details_information, embeddings)
retriever = vectorstore.as_retriever()

/home/rnd-pc/miniconda3/envs/llm/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [165]:

# 4. Define a few short examples for Few-Shot Learning
few_shot_examples = """
Example 1:
User Request: Show all import PI details along with the supplier name, LC number, and expected LC date.
SQL Query: 
SELECT 
    import_pi.id AS import_pi_id,
    import_pi.imp_pi_number, 
    import_pi.imp_pi_date, 
    supplier.name AS supplier_name, 
    import_lc.lc_number, 
    import_lc.lc_opening_date, 
    IFNULL(DATE_SUB(import_pi.etd_date_china, INTERVAL 7 DAY), DATE_ADD(import_pi.etd_date_china, INTERVAL 21 DAY)) AS expected_lc_date
FROM import_pi
LEFT JOIN supplier ON import_pi.supplier_id = supplier.id
LEFT JOIN imp_lc_pi ON imp_lc_pi.import_pi_id = import_pi.id
LEFT JOIN import_lc ON import_lc.id = imp_lc_pi.import_lc_id;

Example 2:
User Request: Display all shipment details for each Import CI along with its invoice number and shipment status.
SQL Query: 
SELECT 
    import_shipment.id AS shipment_id,
    import_shipment.import_ci_id, 
    import_ci.invoice_no, 
    import_shipment.actual_ship_date, 
    import_shipment.eta_ctg_date, 
    IF(inventory.mark_as_full_shipment = 1, 'Full Shipment', 'Partial Shipment') AS shipment_status
FROM import_shipment
LEFT JOIN import_ci ON import_shipment.import_ci_id = import_ci.id
LEFT JOIN inventory ON inventory.import_pi_id = import_shipment.import_ci_id;

Example 3:
User Request: List all import LC details, including their associated suppliers, LC value, balance value, and status.
SQL Query: 
SELECT 
    import_lc.id AS import_lc_id,
    supplier.name AS supplier_name, 
    import_lc.lc_number, 
    import_lc.lc_value, 
    import_lc.lc_balance_value, 
    imp_lc_status.imp_lc_status
FROM import_lc
LEFT JOIN supplier ON import_lc.supplier_id = supplier.id
LEFT JOIN imp_lc_status ON imp_lc_status.id = import_lc.imp_lc_status_id;

Example 4:
User Request: Show the payment details for each supplier along with the CI invoice number and payment amount.
SQL Query: 
SELECT 
    supplier.name AS supplier_name,
    import_ci.invoice_no, 
    supplier_payment.payment_amount, 
    supplier_payment.paid_on_date
FROM supplier_payment
LEFT JOIN supplier ON supplier_payment.supplier_id = supplier.id
LEFT JOIN import_ci ON supplier_payment.import_ci_id = import_ci.id;

Example 5:
User Request: Generate a report of all inventory transactions including the PI number, supplier, and product details.
SQL Query: 
SELECT 
    inventory.id AS inventory_id,
    import_pi.imp_pi_number, 
    supplier.name AS supplier_name, 
    inventory.lot_number, 
    inventory.inventory_date
FROM inventory
LEFT JOIN import_pi ON inventory.import_pi_id = import_pi.id
LEFT JOIN supplier ON import_pi.supplier_id = supplier.id;

Example 6:
User Request: Get a detailed view of import permits along with the supplier name, permit number, and issue date.
SQL Query: 
SELECT 
    import_permit.id AS permit_id, 
    supplier.name AS supplier_name, 
    import_permit.ip_number, 
    import_permit.ip_issue_date_from_depz
FROM import_permit
LEFT JOIN supplier ON import_permit.supplier_id = supplier.id;

Example 7:
User Request: Retrieve all custom clearance details along with the associated supplier and clearance status.
SQL Query: 
SELECT 
    custom_clearance.id AS clearance_id, 
    supplier.name AS supplier_name, 
    custom_clearance.goods_released_date_from_custom, 
    custom_clearance.cust_clearance_status_id
FROM custom_clearance
LEFT JOIN supplier ON custom_clearance.supplier_id = supplier.id;

Example 8:
User Request: Display all import PI records that are marked as full shipment along with the supplier and PI date.
SQL Query: 
SELECT 
    import_pi.imp_pi_number, 
    import_pi.imp_pi_date, 
    supplier.name AS supplier_name
FROM import_pi
LEFT JOIN supplier ON import_pi.supplier_id = supplier.id
LEFT JOIN inventory ON inventory.import_pi_id = import_pi.id
WHERE inventory.mark_as_full_shipment = 1;
"""

meta_data_no_quotes = """
{
  tables: {
    import_pi: {
      columns: {
        id: integer,
        unique_id: string,
        imp_pi_number: string,
        revision_date: date,
        imp_pi_rev_type_id: integer,
        imp_pi_date: date,
        supplier_id: integer,
        cfr_value: decimal,
        freight_charge: decimal,
        fob: decimal,
        etd_date_china: date,
        imp_pi_status_id: integer,
        cancellation_date: date,
        cancellation_reason: string,
        import_flow_status_id: integer,
        isCompensationAdjusted: boolean,
        total_revision_count: integer,
        submitted_to_commercial: boolean,
        created_at: datetime,
        created_by: integer,
        updated_at: datetime,
        updated_by: integer
      },
      primary_key: [id],
      foreign_keys: {
        supplier_id: {
          references: {
            table: supplier,
            column: id
          }
        },
        imp_pi_rev_type_id: {
          references: {
            table: imp_pi_rev_type,
            column: id
          }
        },
        imp_pi_status_id: {
          references: {
            table: imp_pi_status,
            column: id
          }
        },
        import_flow_status_id: {
          references: {
            table: import_flow_status,
            column: id
          }
        }
      },
      description: Contains information about Import Proforma Invoices (PIs), including details such as supplier, status, and financials.
    },
    supplier: {
      columns: {
        id: integer,
        name: string,
        short_name: string,
        address: string,
        active: boolean,
        contact_person: string,
        contact_number: string,
        email: string,
        website: string,
        province: string,
        country_id: integer,
        port: string,
        enlisted_date: date,
        no_of_factories: integer,
        total_employee: integer,
        annual_revenue: decimal,
        supp_org_type_id: integer,
        created_by: integer,
        created_at: datetime,
        updated_by: integer,
        updated_at: datetime
      },
      primary_key: [id],
      foreign_keys: {
        country_id: {
          references: {
            table: country,
            column: id
          }
        },
        supp_org_type_id: {
          references: {
            table: supp_org_type,
            column: id
          }
        }
      },
      description: Contains information about suppliers including contact details and business details.
    }
  },
  relationships: {
    supplier_import_pi: {
      source_table: import_pi,
      target_table: supplier,
      source_column: supplier_id,
      target_column: id,
      relationship_type: many-to-one,
      description: Each Import Proforma Invoice (PI) references a supplier.
    }
  }
}

"""
# 5. Define the prompt template with examples and retrieved schema
# prompt_template = PromptTemplate(
#     template=f"""
#     You are provided with the following database schema information and user request. Your task is to generate a valid SQL query that only uses the tables and columns listed in the provided schema.

#     Database Schema:
#     {meta_data_no_quotes}
  
#     Ensure that:
#     1. You **only** use tables and columns that exist in the provided schema.
#     2. If the user request cannot be fulfilled with the available schema, respond with "The requested information is not available in the current schema."
#     3. Avoid guessing table or column names that are not explicitly mentioned in the schema.
#     4. Use the most relevant tables and columns to generate the query.

#     Here are some examples of valid SQL queries:
#     {few_shot_examples}

#     User Request: {{request}}
    
#     SQL Query:
#     """,
#     input_variables=["context", "request"]
# )
# meta_data_no_quotes

In [166]:
model = "llama3:latest" # llama3:70b | llama3:latest
#llm = OllamaLLM(model=model)
llm = ChatOpenAI(model_name="gpt-4o", openai_api_key=open_ai_key)

In [167]:
details_schema = """ tables,columns,primary_key,foreign_keys,description,relationships,source_table,target_table,source_column,target_column,relationship_type
import_pi,"id: integer, unique_id: string, imp_pi_number: string, revision_date: date, imp_pi_rev_type_id: integer, imp_pi_date: date, supplier_id: integer, cfr_value: decimal, freight_charge: decimal, fob: decimal, etd_date_china: date, imp_pi_status_id: integer, cancellation_date: date, cancellation_reason: string, import_flow_status_id: integer, isCompensationAdjusted: boolean, total_revision_count: integer, submitted_to_commercial: boolean, created_at: datetime, created_by: integer, updated_at: datetime, updated_by: integer","id","supplier_id: references table: supplier column: id, imp_pi_rev_type_id: references table: imp_pi_rev_type column: id, imp_pi_status_id: references table: imp_pi_status column: id, import_flow_status_id: references table: import_flow_status column: id","Contains information about Import Proforma Invoices (PIs), including details such as supplier, status, and financials.",supplier_import_pi,import_pi,supplier,supplier_id,id,many-to-one
supplier,"id: integer, name: string, short_name: string, address: string, active: boolean, contact_person: string, contact_number: string, email: string, website: string, province: string, country_id: integer, port: string, enlisted_date: date, no_of_factories: integer, total_employee: integer, annual_revenue: decimal, supp_org_type_id: integer, created_by: integer, created_at: datetime, updated_by: integer, updated_at: datetime","id","country_id: references table: country column: id, supp_org_type_id: references table: supp_org_type column: id","Contains information about suppliers including contact details and business details.",supplier_import_pi,import_pi,supplier_id,id,many-to-one
"""
text = f"""You are provided with the following database schema information and user request. Your task is to generate a valid SQL query that only uses the tables and columns listed in the provided schema.

    Database Schema:
    {schema_documents}

    Ensure that:
    1. You **only** use tables and columns that exist in the provided schema.
    2. If the user request cannot be fulfilled with the available schema, respond with "The requested information is not available in the current schema."
    3. Avoid guessing table or column names that are not explicitly mentioned in the schema.
    4. Use the most relevant tables and columns to generate the query.

    Here are some examples of valid SQL queries:
    {few_shot_examples}"""


prompt_template = PromptTemplate(
    template=f"""
    {text}

    User Request: {{request}}
    
    SQL Query:
    """,
    input_variables=["context", "request"]
)


In [185]:

# 7. Create a function to generate SQL queries with RAG
def generate_sql_query_with_rag(user_request):
    # Retrieve relevant schema information based on the user request
    retrieved_docs = retriever.get_relevant_documents(user_request)
   
    # Use the correct attribute to access document content
    relevant_context = "\n".join([doc.page_content for doc in retrieved_docs])
    # Create the complete prompt with retrieved context
    prompt = prompt_template.format(context=relevant_context, request=user_request)
    
    # Generate the SQL query
    sql_query = llm.invoke(prompt)
    
    return sql_query


In [169]:
from sqlalchemy.sql import text  # Import the text function

# 8. Execute the generated SQL query
def execute_sql_query(sql_query):
    # Check and debug: Ensure sql_query is a string
    if not isinstance(sql_query, str):
        return f"Error: The provided SQL query is not a string. Got {type(sql_query)}"

    try:
        with engine.connect() as connection:
            # Use the text() function to convert the string query into an executable object
            result = connection.execute(text(sql_query))
            return result.fetchall()
    except SQLAlchemyError as e:
        print("*****SQL Error****")
        return str(e)

In [170]:
# 9. Combine everything into a LangChain workflow
# sql_generation_chain = LLMChain(
#     prompt=prompt_template,
#     llm=llm
# )
sql_generation_chain = prompt_template | llm

In [171]:
import re

def extract_sql_query(text):
    # Define a regular expression pattern to match the SQL query between triple backticks (```sql ... ```)
    pattern = r"```sql(.*?)```"
    match = re.search(pattern, text, re.DOTALL)  # re.DOTALL to match across multiple lines
    
    if match:
        # Extract the SQL query from the match
        sql_query = match.group(1).strip()
        # Remove any newline and extra whitespace
        formatted_query = " ".join(sql_query.split())
        return formatted_query
    else:
        return None

In [208]:
# Example usage with RAG
user_prompt = "Generate a report of all inventory transactions including the PI number, supplier, and product details."
IS_LLAMA = False
generated_sql = generate_sql_query_with_rag(user_prompt)
# generated_sql

if IS_LLAMA:
# -------------- For another model ----------------------
    query_results = execute_sql_query(generated_sql)
    query = extract_sql_query(query_results)
# --------------- End another model ------------
else:
# ------------- For Open AI -------------
    #query = generated_sql.content.strip() # for turbo
    query = extract_sql_query(generated_sql.content)

# Print the results
#print("Generated SQL Query:\n", query_results)
print("----------------------------------------")
print("Query:\n", query)
print("----------------------------")

#q = extract_sql_query(generated_sql.content)
result = execute_sql_query(query)
result

retrieve docs  q
q
q
q
----------------------------------------
Query:
 SELECT inventory.id AS inventory_id, import_pi.imp_pi_number, supplier.name AS supplier_name, inventory.lot_number, inventory.inventory_date, product.article AS product_details FROM inventory LEFT JOIN import_pi ON inventory.import_pi_id = import_pi.id LEFT JOIN supplier ON import_pi.supplier_id = supplier.id LEFT JOIN imp_pi_prod_dtl ON imp_pi_prod_dtl.import_pi_id = import_pi.id LEFT JOIN product ON imp_pi_prod_dtl.product_id = product.id;
----------------------------


[(1, None, None, 'OI-002/21', datetime.date(2017, 7, 1), None),
 (2, None, None, 'OI-011/17', datetime.date(2017, 7, 1), None),
 (3, None, None, 'OI-011/17', datetime.date(2017, 7, 1), None),
 (4, None, None, 'OI-011/17', datetime.date(2017, 7, 1), None),
 (5, None, None, 'OI-024/17', datetime.date(2017, 7, 1), None),
 (6, None, None, 'OI-024/17', datetime.date(2017, 7, 1), None),
 (7, None, None, 'OI-024/17', datetime.date(2017, 7, 1), None),
 (8, None, None, 'OI-039/17', datetime.date(2017, 7, 1), None),
 (9, None, None, 'OI-046/17', datetime.date(2017, 7, 1), None),
 (10, None, None, 'OI-046/17', datetime.date(2017, 7, 1), None),
 (11, None, None, 'OI-046/17', datetime.date(2017, 7, 1), None),
 (12, None, None, 'OI-054/21', datetime.date(2017, 7, 1), None),
 (13, None, None, 'OI-068/17', datetime.date(2017, 7, 1), None),
 (14, None, None, 'OI-080/17', datetime.date(2017, 7, 1), None),
 (15, None, None, 'OI-081/17', datetime.date(2017, 7, 1), None),
 (16, None, None, 'OI-084/17', dat

In [173]:
def visulize_result(sql, result):
    # Define the prompt template
    template = """
    You are a data visualization assistant. You will receive a SQL query along with its result data in Python list format.
    Your task is to understand the structure of the result and display it in a clear tabular format.

    Here is the SQL query:
    {sql_query}

    And here is the result data in Python list format:
    {sql_result}

    Create a well-formatted table to visualize the given data. Use headers as necessary.
    """
    
    # Create the prompt template object
    prompt_template = PromptTemplate(template=template, input_variables=["sql_query", "sql_result"])
    
    # Initialize the language model with your specified parameters
    llm = ChatOpenAI(model_name="gpt-4", openai_api_key=open_ai_key) # Replace "gpt-4" with "gpt-4o" if needed
    
    # Create the LLM chain
    llm_chain = prompt_template | llm
    
    # Format the input for the LLM chain
    formatted_input = {
        "sql_query": sql,
        "sql_result": result
    }
    
    # Execute the LLM chain with the formatted input
    output = llm_chain.invoke(formatted_input)
    
    return output


In [174]:
# Define the user request
# user_request = user_prompt

# # Run the LLMChain with the provided user request
# generated_sql = sql_generation_chain.invoke({
#     "context": "Relevant schema information here, e.g., columns and table structure",
#     "request": user_request
# })
# #print(generated_sql['text'])
# # Print the generated SQL query
# qry = extract_sql_query(generated_sql['text'])
# result = execute_sql_query(qry)
# print(qry,"\n--------------")
# result